In [ ]:
import os
import shutil
import zipfile
import pandas as pd
import numpy as np
import rasterio
from pyproj import CRS
from affine import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling

import matplotlib.pyplot as plt
import re

In [ ]:
mc_path = '../run_data/results'
outpath = os.path.join(mc_path, 'postproc')
proc_heads = True


In [ ]:
if not os.path.exists(outpath):
    os.mkdir(outpath)

In [ ]:
priority_wells = ['WWR0000401_Davos Well #3',
                  'WWR0000401_Riverside Well #3',
                  'WWR0000479_Center St. Well',
                  'WWR0000479_Fallsview Well',
                  'WWR0000479_Harris Well 1',
                  'WWR0000479_Harris Well 2',
                  'WWR0001055_Mountaindale Well #2',
                  'WWR0001748_Well 3']

well_nums = [i.replace(" ", "_").replace("#", "") for i in priority_wells ]

dest_nodes = [1042614, 214382, 562588, 574351, 575587, 574968, 1041449, 344623]


node_dict = dict(zip(dest_nodes, well_nums))

In [ ]:
wells_df = pd.read_csv('../processed_input/2009-2016_ave_pumping.csv')
wells_df = wells_df.loc[wells_df.WellType == 'Priority']
priority_wells = wells_df.ID_Well.tolist()

In [ ]:
well_dat_df = pd.read_csv('../neversink_mf6/wel_000.dat', delim_whitespace=True)
well_dat_df = well_dat_df.loc[well_dat_df.boundname.isin(priority_wells)]

well_dat_df['i_zb'] = well_dat_df['i'] - 1
well_dat_df['j_zb'] = well_dat_df['j'] - 1

particle_ys = [y for y in well_dat_df['i_zb']]
particle_xs = [x for x in well_dat_df['j_zb']]

In [ ]:
#  set true to make a single combined probabilistic capture geotiff - combines capture for all wells
make_combined=True

#  set true to make 8 probabilistic capture geotiffs - one for each well
make_wells=True

In [ ]:
mp_grid = np.zeros((680, 619))
heads = {}
nonzero_mc_count = 0

zip_files = [i for i in os.listdir(mc_path) if i.endswith('zip')]

for file in zip_files:
    crun = int(re.findall('\d+',file)[0])
    print('processing run # {}\r'.format(crun), end='')
    #  check to make sure zip files are non-zero in size
    if os.stat(os.path.join(mc_path,file)).st_size == 0:
        print(f"{file} file has zero size -- skipping file")
        continue

    #  if zip file is greater than zero, increase non-zero tally
    nonzero_mc_count +=1

    # unzip result zip files into postproc folder
    with zipfile.ZipFile(os.path.join(mc_path, file), 'r') as zip_ref:
        zip_ref.extractall(outpath)

    #  read results into memory 
    results = [i for i in os.listdir(outpath) if i.startswith('endpoint_cells')]
    for result in results:
        df = pd.read_csv(os.path.join(outpath, result))

        i_coords = df.j.tolist()
        j_coords = df.i.tolist()

        for i, j in zip(i_coords, j_coords):
            mp_grid[j, i] += 1

    if proc_heads:
        heads[crun] = np.loadtxt(os.path.join(outpath, 'wt_array.txt'))
     #  clean up unzipped results files
 #   files_to_remove = [file for file in os.listdir(outpath) if file.endswith('csv')]
 #   for ftr in files_to_remove:
 #       os.remove(os.path.join(outpath, ftr))
   # break

In [ ]:
heads

In [ ]:
allh = np.array([i for _,i in heads.items()])

In [ ]:
if proc_heads is True:
    h_std = allh.std(axis=0)
    h_std[h_std>1e10]=-9999.
   
    outfile = 'heads_std_tmp.tif'
    raster_outpath = os.path.join(outpath, outfile)

    dataset = rasterio.open(
        raster_outpath, 
        'w', 
        driver='GTiff',
        height = h_std.shape[0], 
        width = h_std.shape[1],
        count=1,
        nodata = -9999.0,
        dtype=str(h_std.dtype),
        crs = CRS.from_epsg(5070),
        transform=Affine(50.0, 0.0, 1742955.0,
                         0.0, -50.0, 2292285.0),
    )

    dataset.write(h_std, 1)
    dataset.close()

In [ ]:
dst_crs = CRS.from_epsg(4326)
with rasterio.open(raster_outpath) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': CRS.from_epsg(4326),
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open('heads_tiff_4326.tif', 
                      'w', **kwargs) as dest:
        reproject(
            source = h_std,
            destination = rasterio.band(dest,1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )


h_std[h_std<-999]= np.nan
plt.imshow(h_std)
plt.colorbar()

In [ ]:
h_std

In [ ]:
transform

In [ ]:
outpath

In [ ]:
if make_combined:
    
    mp_grid = np.zeros((680, 619))
    nonzero_mc_count = 0

    zip_files = [i for i in os.listdir(mc_path) if i.endswith('zip')]

    for file in zip_files:
        #  check to make sure zip files are non-zero in size
        if os.stat(os.path.join(mc_path,file)).st_size == 0:
            print(f"{file} file has zero size -- skipping file")
            continue
        
        #  if zip file is greater than zero, increase non-zero tally
        nonzero_mc_count +=1
        print(f'{nonzero_mc_count}\r', end="")
        # unzip result zip files into postproc folder
        with zipfile.ZipFile(os.path.join(mc_path, file), 'r') as zip_ref:
            zip_ref.extractall(outpath)

        #  read results into memory 
        results = [i for i in os.listdir(outpath) if i.startswith('endpoint_cells')]
        for result in results:
            df = pd.read_csv(os.path.join(outpath, result))

            i_coords = df.i.tolist()
            j_coords = df.j.tolist()

            for i, j in zip(i_coords, j_coords):
                mp_grid[i, j] += 1

         #  clean up unzipped results files
        files_to_remove = [file for file in os.listdir(outpath) if file.endswith('csv')]
        for ftr in files_to_remove:
            os.remove(os.path.join(outpath, ftr))

    print(f'\nNumer of successful MODPATH runs: {nonzero_mc_count}')
    #  divide modpath grid by number of successful ensemble members to get probabalistic coverage
    mp_grid /= nonzero_mc_count

    #  plot results with priority wells
    grid_for_plotting = mp_grid.copy()
    grid_for_plotting[grid_for_plotting == 0.]=np.nan

    plt.figure(figsize=(10,10))
    plt.imshow(grid_for_plotting)
    plt.colorbar(shrink=0.8)
    plt.scatter(particle_xs, particle_ys, c='r', label='Priority Well')
    plt.title('Probabilistic')
    plt.legend()
    plt.show()
    
    #  export results as geotiff
    out_grid = mp_grid.copy()
    out_grid[out_grid == 0.] = -9999.
    
    outfile = 'mp_mc_combined.tif'
    raster_outpath = os.path.join(outpath, outfile)

    dataset = rasterio.open(
        raster_outpath, 
        'w', 
        driver='GTiff',
        height = out_grid.shape[0], 
        width = out_grid.shape[1],
        count=1,
        nodata = -9999.0,
        dtype=str(out_grid.dtype),
        crs = CRS.from_epsg(5070),
        transform=Affine(50.0, 0.0, 1742955.0,
                         0.0, -50.0, 2292285.0),
    )

    dataset.write(out_grid, 1)
    dataset.close()

In [ ]:
np.unique(mp_grid)

In [ ]:
if make_wells:
    for node in dest_nodes:
        mp_grid = np.zeros((680, 619))
        nonzero_mc_count = 0

        zip_files = [i for i in os.listdir(mc_path) if i.endswith('zip')]

        for file in zip_files:
            #  check to make sure zip files are non-zero in size
            if os.stat(os.path.join(mc_path,file)).st_size == 0:
                print(f"{file} file has zero size -- skipping file")
                continue

            # unzip result zip files into postproc folder
            with zipfile.ZipFile(os.path.join(mc_path, file), 'r') as zip_ref:
                zip_ref.extractall(outpath)

            #  read results into memory 
            results = [i for i in os.listdir(outpath) if i.startswith('endpoint_cells')]
            for result in results:
                df = pd.read_csv(os.path.join(outpath, result))
                
                #  extract particles for well of interest
                df = df.loc[df.node == node]
                
                if df.shape[0] == 0:
                    continue
                
                #  if dataframe contains well results, increase non-zero tally
                nonzero_mc_count +=1

                i_coords = df.i.tolist()
                j_coords = df.j.tolist()

                for i, j in zip(i_coords, j_coords):
                    mp_grid[i, j] += 1

             #  clean up unzipped results files
            files_to_remove = [file for file in os.listdir(outpath) if file.endswith('csv')]
            for ftr in files_to_remove:
                os.remove(os.path.join(outpath, ftr))

        print(f'\nNumer of successful MODPATH runs: {nonzero_mc_count}')
        #  divide modpath grid by number of successful ensemble members to get probabalistic coverage
        mp_grid /= nonzero_mc_count

        #  plot results with priority wells
        grid_for_plotting = mp_grid.copy()
        grid_for_plotting[grid_for_plotting == 0.]=np.nan

        plt.figure(figsize=(10,10))
        plt.imshow(grid_for_plotting)
        plt.colorbar(shrink=0.8)
        plt.scatter(particle_xs, particle_ys, c='r', label='Priority Well')
        plt.title('Probabilistic capture at {}'.format(node_dict[node]))
        plt.legend()
        plt.show()

        #  export results as geotiff
        out_grid = mp_grid.copy()
        out_grid[out_grid == 0.] = -9999.

        outfile = 'mp_mc_{}.tif'.format(node_dict[node])
        raster_outpath = os.path.join(outpath, outfile)

        dataset = rasterio.open(
            raster_outpath, 
            'w', 
            driver='GTiff',
            height = out_grid.shape[0], 
            width = out_grid.shape[1],
            count=1,
            nodata = -9999.0,
            dtype=str(out_grid.dtype),
            crs = CRS.from_epsg(5070),
            transform=Affine(50.0, 0.0, 1742955.0,
                             0.0, -50.0, 2292285.0),
        )

        dataset.write(out_grid, 1)
        dataset.close()

In [ ]:
np.unique(out_grid)